<a href="https://colab.research.google.com/github/ufrpe-mineracao-textos/projeto-de-mineracao-20192-automatic-essay-evaluation/blob/master/Encontrando_linhas_a_partir_da_intensidade_m%C3%A9dia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ufrpe-mineracao-textos/projeto-de-mineracao-20192-automatic-essay-evaluation.git

Cloning into 'projeto-de-mineracao-20192-automatic-essay-evaluation'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 118 (delta 13), reused 54 (delta 7), pack-reused 53
Receiving objects: 100% (118/118), 53.14 MiB | 37.39 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [0]:
import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow

In [0]:
# Loading essay images
images_folder  = "projeto-de-mineracao-20192-automatic-essay-evaluation/data/essay_pictures/"
essays = {}
for r,d,files in os.walk(images_folder):
    files.sort()
    for f in files:
        essays[f] = cv2.imread(os.path.join(r,f))

# Encontrando as linhas

In [0]:
# As funções calculam a intensidades média para cada linha ou coluna da imagem
# e retornam as possíveis linhas horizontais ou verticais baseada no limiar informado.
# Para obter-se o resultado esperado pode ser necessário ordenar todas as linhas
# obtidas baseado em alguma característica, como espessura ou posição.

def verticalLines(image, thresh, totalLines=None, sort='thickness',reverse=True):
    vals = np.array([s for s in [np.sum(image[:,y])/(255*image.shape[0]) for y in range(image.shape[1])]])
    lines = []
    y = 0
    while y < image.shape[1]:
        if vals[y] < thresh:
            start = y
            y += 1
            while y < image.shape[1] - 1 and vals[y] < thresh:
                y += 1
            l = {}
            l['pos'] = (start,y)
            l['thickness'] = y - start
            l['mean'] = np.mean(vals[start:y])
            lines.append(l)
        y += 1
    if sort == 'thickness':
        lines.sort(key = lambda x: x[sort],reverse=reverse)
    else:
        lines.sort(key = lambda x: x[sort],reverse=reverse)
    if totalLines == None:
        return lines
    else:
        return lines[:min(totalLines,len(lines))]

def horizontalLines(image, thresh, totalLines=None, sort='thickness', reverse=True):
    vals = np.array([s for s in [np.sum(image[x,:])/(255*image.shape[1]) for x in range(image.shape[0])]])
    lines = []
    x = 0
    while x < image.shape[0]:
        if vals[x] < thresh:
            start = x
            x += 1
            while x < image.shape[0] - 1 and vals[x] < thresh:
                x += 1
            l = {}
            l['pos'] = (start,x)
            l['thickness'] = x  - start
            l['mean'] = np.mean(vals[start:x])
            lines.append(l)
        x += 1
    if sort == 'thickness':
        lines.sort(key = lambda x: x[sort], reverse=reverse)
    else:
        lines.sort(key = lambda x: x[sort],reverse=reverse)
    if totalLines == None:
        return lines
    else:
        return lines[:min(totalLines,len(lines))]

In [0]:
def findLines(essays):
    # Tentando detectar as posições das 4 linhas mais grossas
    LINES = {}
    for e in essays:
        gray = cv2.cvtColor(essays[e],cv2.COLOR_BGR2GRAY)
        for t in np.arange(0.4,0.9,0.1):
            hlines = horizontalLines(gray, t, 2)
            if len(hlines) == 2:
                break
        if len(hlines) != 2:
            print('Linhas horizontais delimitantes não encontradas ('+str(len(hlines))+'/'+'2)')
            print('essay: '+e)
            return {}
        for t in np.arange(0.4,0.9,0.1):
            vlines = verticalLines(gray,t, 2)
            if len(vlines) == 2:
                break
        if len(vlines) != 2:
            print('Linhas verticais delimitantes não encontradas ('+str(len(vlines))+'/'+'2)')
            print('essay: '+e)
            return {}
        
        LINES[e] = {}
        LINES[e]['vertical'] = vlines
        LINES[e]['horizontal'] = hlines

    # Tentando detectar o restante
    for e in essays:
        gray = cv2.cvtColor(essays[e],cv2.COLOR_BGR2GRAY)
        evlines = [v['pos'] for v in LINES[e]['vertical']]
        evlines.sort()
        #print(evlines)
        # tenta cortar a numeração
        cropV = int(np.round((evlines[1][0] - evlines[0][1]) / 50))
        start = evlines[0][1]+cropV
        end = evlines[1][0]-cropV
        for t in np.arange(0.4,0.9,0.1):
            vlines = verticalLines(gray[:,start:end],t, 1)
            for i in range(len(vlines)):
                t = vlines[i]['pos']
                vlines[i]['pos'] = (t[0]+start,t[1]+start)
            if len(vlines) == 1:
                break
        if len(vlines) != 1:
            print('Linhas verticais não encontradas (0/1)')
            print('essay: '+e)
            return {}
        
        LINES[e]['vertical'] += vlines
        LINES[e]['vertical'].sort(key=lambda a: a['pos'][0])

        ehlines = [h['pos'] for h in LINES[e]['horizontal']]
        ehlines.sort()
        
        
        # procura as linhas horizontais de uma em uma
        pixels_per_line = (ehlines[1][0]-ehlines[0][1]) / 30
        cropH = int(pixels_per_line/4)
        
        for l in range(29):
            for t in np.arange(0.4,0.9,0.1):
                start = ehlines[0][1] + int(l * pixels_per_line) + cropH
                end = start + int(pixels_per_line) #- cropH
                hlines = horizontalLines(gray[start:end,evlines[0][1]+cropV:vlines[0]['pos'][1]], t, 1, sort='pos', reverse=True)
                hlines.sort(key=lambda a: a['pos'], reverse=True)
                hlines = hlines[:1]
                for i in range(len(hlines)):
                    t = hlines[i]['pos']
                    hlines[i]['pos'] = (t[0]+start,t[1]+start)
                if len(hlines) == 1:
                        break
            if len(hlines) != 1:
                print('Linhas horizontais não encontradas ('+str(l)+'/'+'29)')
                print('essay: '+e)
                return {}
            LINES[e]['horizontal'] += hlines
        LINES[e]['horizontal'].sort(key=lambda a: a['pos'][0])
        
    return LINES

In [0]:
LINES = findLines(essays)

# Salva imagens com linhas encontradas

In [0]:
# desenha linhas na imagem
def imshow_lines(image, horizontalLines, verticalLines, file=None):
    img = np.copy(image)
    if horizontalLines != None and len(horizontalLines) != 0:
        y1 = horizontalLines[0][0]
        y2 = horizontalLines[-1][1]
    else:
        y1 = 0
        y2 = image.shape[0]
    if verticalLines != None and len(verticalLines) != 0:
        for vl in verticalLines:
            x = vl[0]
            thickness = max(vl[1] - x,2)
            cv2.line(img, (x,y1), (x,y2), (0,0,255), thickness)
        x1 = verticalLines[0][0]
        x2 = verticalLines[-1][1]
    else:
        x1 = 0
        x2 = img.shape[1]
    if horizontalLines != None and len(verticalLines) != 0:
        for hl in horizontalLines:
            y = hl[0]
            thickness = max(hl[1] - y,3)
            cv2.line(img, (x1,y), (x2,y), (0,0,255), thickness)
    if file != None:
        cv2.imwrite(file, img)
    else:
        cv2_imshow(img)

In [0]:
for e in essays:
    horl = [h['pos'] for h in LINES[e]['horizontal']]
    verl = [v['pos'] for v in LINES[e]['vertical']]
    imshow_lines(essays[e], horl, verl , e)